##  ‧₊˚🎄✩ ₊˚🦌⊹♡ Advent of Code 2024 ‧₊˚🎄✩ ₊˚🦌⊹♡

The resolutions for the new year are to have a good trade off between shortness, readability and performance

In [2]:
import operator
import itertools
import functools
import collections
import numpy as np
import pandas as pd

----------

### Day 1: Historian Hysteria 🎄

Easy start. Using `pandas` makes the import a bit cleaner to me in this first game.

In [10]:
df = pd.read_csv('data/day1.txt', sep='   ', header=None, engine='python')

delta = np.abs(np.sort(df[0]) - np.sort(df[1])).sum()
counter = collections.Counter(df[1])
delta_diff = sum(i * counter[i] for i in df[0])

print(f"Answer 1: {delta}")
print(f"Answer 2: {delta_diff}")

Answer 1: 2375403
Answer 2: 23082277


-----------

### Day 2: Red-Nosed Reports 🦌🔴

Less clean than the solution proposed for instance by Peter Norvig but more efficient. Also, in `is_safe_repair`, Peter tries all the combinations while just the first one is enough because the texts says it is feasible if we can repair ONE infeasibility. But it's okay, Google is Google and capitalism is 💩.

In [17]:
with open ('data/day2.txt') as f:
    rows = [list(map(int, line.split(' '))) for line in f]

def inrange(delta):
    return 1 <= abs(delta) <= 3

def is_safe(row):
    rowthread = int(row[0] > row[-1])
    return all(int(i > j) == rowthread and inrange(i - j) for i, j in zip(row, row[1:]))

def is_safe_repair(row):
    rowthread = int(row[0] > row[-1])
    idx = next(idx for idx, (i, j) in enumerate(zip(row, row[1:])) if int(i > j) != rowthread or not inrange(i - j))
    return is_safe(row[:idx] + row[idx + 1:]) or is_safe(row[:idx + 1] + row[idx + 2:])

safe_count = sum(1 for row in rows if is_safe(row))
safe_repair = sum(1 for row in rows if is_safe(row) or is_safe_repair(row))

print("Answer 1: ", safe_count)
print("Answer 2: ", safe_repair)

Answer 1:  356
Answer 2:  413


-------

### Day 3: Mull It Over 📖

The book in the title is because we are doing string manipulation, the most horrible of all tasks. Together with carbon for bad kids ⛽️ because I don't want to lose time on this. The `import re` is here because I hope I don't need regex anymore.

Curious that `mul` is also a function in the `operator` module, so we can use `eval('operator.' + mulstr)` after importing `operator` to evaluate a multiplication ⚡️.

I'm definetly not an expert, but it seems regex multiline is not working, that's why `text.replace('\n', '')`. I like the 💄 emoji.

Please note this solution is not working in case of a deactivation `don't()` at the end not followed by any `do()`, but I verified this was not the case in the dataset and, again, I hate losing time on strings manipulation.

In [4]:
import re

with open ('data/day3.txt') as f:
    text = f.read()

def compile(mulstr):
    return eval('operator.' + mulstr)

total = sum(compile(i) for i in re.findall(r'mul\(\d+,\d+\)', text))
active_text = re.sub(r"don't\(\).*?(do\(\)|$)", "💄", text.replace('\n', ''))
active_total = sum(compile(i) for i in re.findall(r'mul\(\d+,\d+\)', active_text))

print("Answer 1: ", total)
print("Answer 2: ", active_total)

Answer 1:  184576302
Answer 2:  118173507


--------------

### Day 4: Ceres Search 🍺

As italian, the Ceres to me is a beer 🍺. And the input is a beautiful 140 x 140 matrix, and to me this means `numpy`.
Curious that I wrote the first function `get_xmas_in_line` to detect horizontal written XMAS and I realized, first, that it was possible to use it for vertically written XMAS by simply transposing the matrix, then, that it was possible to use it for diagonal XMAS as well 🤯 by rolling separately each column. However, it was getting too complicated and I opted for a simple nested for loop 🫏.

Like `a` becomes `b` by rolling column 0 by 0, column 1 by -1, column 2 by -2 and column 3 by -3.
```
a = [[X # # #]
     [# M # #]
     [# # A #]
     [# # # S]]
b = [[X M A S]
     [# # # #]
     [# # # #]
     [# # # #]]
```

Second part on X-MAS reminds me a porn (that's why `get_xmas_diagonal_porn`function).

Horrible idea using `numpy`, I give myself a 5 this time. Julia functions vectorization is way cleaner by the way.

In [66]:
with open ('data/day4.txt') as f:
    data = np.array([list(line.replace('\n', '')) for line in f])

def get_xmas_in_line(matrix, rolling):
    xmas, samx = np.array(['X', 'M', 'A', 'S']), np.array(['S', 'A', 'M', 'X'])
    rolled_matrix = np.roll(matrix, rolling, axis=1)
    rolled_matrix[:, :rolling] = "N"
    return int(np.all( rolled_matrix.reshape((-1, 4)) == xmas, axis=1).astype(int).sum()) + \
           int(np.all( rolled_matrix.reshape((-1, 4)) == samx, axis=1).astype(int).sum())

def get_numeric_matrix(matrix):
    num = np.zeros(matrix.shape)
    for letter, number in zip(('X', 'M', 'A', 'S'), range(1, 5)):
        num[matrix == letter] = number
    return num

def get_xmas_diagonal(matrix):
    xmas, samx = np.array([1,2,3,4]), np.array([4,3,2,1])
    return sum(
        sum([
            int(np.array_equal((matrix[i:i+4, j:j+4] * np.eye(4,4)).sum(axis=0), xmas)),
            int(np.array_equal((matrix[i:i+4, j:j+4] * np.eye(4,4)).sum(axis=0), samx)),
            int(np.array_equal((matrix[i:i+4, j:j+4] * np.flip(np.eye(4,4), axis=1)).sum(axis=0), xmas)),
            int(np.array_equal((matrix[i:i+4, j:j+4] * np.flip(np.eye(4,4), axis=1)).sum(axis=0), samx))
        ])
        for i in range(matrix.shape[0] - 3) for j in range(matrix.shape[1] - 3)
    )

def get_xmas_diagonal_porn(matrix):
    xeye = np.array([[1, 0, 1], [0, 1, 0], [1, 0, 1]])
    return sum(
        sum([
            int(np.array_equal( matrix[i:i+3, j:j+3] * xeye, np.array([[2, 0, 2],[0, 3, 0],[4, 0, 4]]))),
            int(np.array_equal( matrix[i:i+3, j:j+3] * xeye, np.array([[4, 0, 4],[0, 3, 0],[2, 0, 2]]))),
            int(np.array_equal( matrix[i:i+3, j:j+3]* xeye, np.array([[4, 0, 2],[0, 3, 0],[4, 0, 2]]))),
            int(np.array_equal( matrix[i:i+3, j:j+3]* xeye, np.array([[2, 0, 4],[0, 3, 0],[2, 0, 4]]))),
        ])
        for i in range(matrix.shape[0] - 2) for j in range(matrix.shape[1] - 2)
    )

h = sum(get_xmas_in_line(data, rolling=i) for i in range(4))
v = sum(get_xmas_in_line(data.T, rolling=i) for i in range(4))
d = get_xmas_diagonal(get_numeric_matrix(data))
xd = get_xmas_diagonal_porn(get_numeric_matrix(data))
print("Answer 1: ", h + v + d)
print("Answer 2: ", xd)

Answer 1:  2358
Answer 2:  1737
